# Task 1

In [1]:
# 1. Model the problem as a state-space graph
museum_graph = {
    'E': ['A', 'B'],
    'A': ['C', 'D'],
    'B': ['D'],
    'C': ['F'],
    'D': ['C', 'G'],
    'F': ['D'],
    'G': ['V'],
    'V': [] # Vault has no outgoing corridors in this problem
}

print("Museum Layout (Directed Graph):")
for room, connections in museum_graph.items():
    print(f"  {room} -> {', '.join(connections) if connections else 'No outgoing corridors'}")


Museum Layout (Directed Graph):
  E -> A, B
  A -> C, D
  B -> D
  C -> F
  D -> C, G
  F -> D
  G -> V
  V -> No outgoing corridors


### 2. Apply suitable algorithm to find a path from E to V. Justify why you used that.

### 3. Clearly show the order of node expansion.

### 4. Justify why BFS is the most appropriate algorithm for this problem.

**Algorithm Choice: Breadth-First Search (BFS)**

BFS is the most appropriate algorithm for this problem because:

*   **Unweighted Graph:** Each corridor traversal takes "exactly one unit of time." This means all edges in the graph have a uniform weight (1). BFS is guaranteed to find the shortest path (in terms of number of edges/moves) in an unweighted graph.
*   **Shortest Path Requirement:** The primary constraint is to reach the Vault (V) "within four moves." Finding the shortest path directly addresses this constraint, as BFS explores the graph layer by layer, guaranteeing that the first time it finds the target node, it has found a path with the minimum number of steps.
*   **Simplicity:** BFS is relatively straightforward to implement and understand for this type of problem.

#### Implementation of BFS:

In [2]:
from collections import deque

def bfs(graph, start, end, max_moves):
    queue = deque([(start, [start], 0)]) # (current_node, path_taken, moves_count)
    visited = {start}

    print(f"\nStarting BFS from {start} to {end} with max {max_moves} moves.")
    print("Order of Node Expansion:")

    while queue:
        current_node, path, moves = queue.popleft()

        print(f"  Expanding: {current_node} (Moves: {moves}, Path: {'>'.join(path)})")

        # Check if we've reached the destination
        if current_node == end:
            if moves <= max_moves:
                return path, moves
            else:
                print(f"  Vault reached, but it took {moves} moves, which is more than the allowed {max_moves}.")
                # Continue searching for a path within limits if possible, or decide this is a failure for current path
                # For shortest path, if found, it's optimal, so we can stop.
                # However, the problem asks for *a* path within 4 moves, so we keep searching if this one is too long.
                # Given BFS finds shortest, if *this* shortest is too long, all other paths will be longer or same length.
                # So, if we reach V here, we know it's the shortest path. If moves > max_moves, no path exists within limits.
                return None, None # No path found within max_moves

        # If we exceed max_moves, stop exploring this path
        if moves >= max_moves:
            continue

        # Explore neighbors
        for neighbor in graph.get(current_node, []):
            if neighbor not in visited:
                visited.add(neighbor)
                new_path = path + [neighbor]
                queue.append((neighbor, new_path, moves + 1))

    return None, None # No path found

start_room = 'E'
end_room = 'V'
max_allowed_moves = 4

shortest_path, total_moves = bfs(museum_graph, start_room, end_room, max_allowed_moves)

if shortest_path:
    print(f"\nPath found from {start_room} to {end_room}: {'>'.join(shortest_path)}")
    print(f"Total moves: {total_moves}")
    print(f"This path meets the '{max_allowed_moves} moves' constraint.")
else:
    print(f"\nNo path found from {start_room} to {end_room} within {max_allowed_moves} moves.")



Starting BFS from E to V with max 4 moves.
Order of Node Expansion:
  Expanding: E (Moves: 0, Path: E)
  Expanding: A (Moves: 1, Path: E>A)
  Expanding: B (Moves: 1, Path: E>B)
  Expanding: C (Moves: 2, Path: E>A>C)
  Expanding: D (Moves: 2, Path: E>A>D)
  Expanding: F (Moves: 3, Path: E>A>C>F)
  Expanding: G (Moves: 3, Path: E>A>D>G)
  Expanding: V (Moves: 4, Path: E>A>D>G>V)

Path found from E to V: E>A>D>G>V
Total moves: 4
This path meets the '4 moves' constraint.


### 5. Prove whether BFS guarantees the optimal solution under the given constraints.

**BFS Guarantees Optimal Solution:**

Yes, for an **unweighted graph** (where each edge traversal has the same cost, in this case, 1 unit of time), Breadth-First Search (BFS) is guaranteed to find the **optimal solution** in terms of the shortest path (minimum number of moves/edges).

**Proof:**

1.  **Layer-by-Layer Exploration:** BFS explores the graph level by level. It first visits all nodes 1 step away from the source, then all nodes 2 steps away, and so on.
2.  **First Encounter is Shortest:** When BFS encounters the target node for the first time, it *must* have found the shortest path. This is because if there were a shorter path, BFS would have discovered it at an earlier level of the search tree, and thus visited the target node sooner.
3.  **No Backtracking for Shorter Paths:** Since all edges have uniform weight, there is no possibility of a longer path (in terms of number of edges) eventually leading to a shorter *total cost* because all costs are equal. Therefore, the first path found is always the one with the fewest edges.

In our museum problem, since each corridor traversal takes "exactly one unit of time," the graph is unweighted. Thus, the BFS algorithm will indeed find the path with the minimum number of corridor traversals (moves) from the Entrance to the Vault, thereby guaranteeing the optimal solution in terms of moves.

### 6. Analyze the time and space complexity of the algorithm.

For a graph represented with adjacency lists (like our dictionary `museum_graph`), where `V` is the number of vertices (rooms) and `E` is the number of edges (corridors):

*   **Time Complexity: O(V + E)**
    *   Each vertex (room) is enqueued and dequeued at most once. The operations for adding/removing from the queue are O(1).
    *   When a vertex is dequeued, we iterate over all its outgoing edges to find its neighbors. In the worst case, every edge in the graph will be examined once (from the perspective of its source vertex).
    *   Therefore, the total time complexity is proportional to the sum of the number of vertices and the number of edges.

*   **Space Complexity: O(V + E)**
    *   **Queue:** In the worst case, the queue can hold all vertices in a particular level of the graph. At most, it can hold O(V) vertices.
    *   **Visited Set:** The `visited` set stores all vertices that have been enqueued, which is at most O(V) vertices.
    *   **Path Storage:** Storing the current path for each node in the queue (e.g., `path` variable in our implementation) can also contribute to space. In the worst case, a path could be O(V) long, and if many such paths are stored in the queue, it could theoretically lead to O(V*E) or worse depending on specific graph structures and if paths are copied. However, a more typical analysis for standard BFS (where you reconstruct the path by parent pointers) would be O(V) for parent pointers and O(V) for the queue/visited set. In our implementation, where we store the full path with each queue item, the space complexity *could* be higher for dense graphs, but for typical use cases, it's still often discussed as O(V+E) if the paths are not excessively long or numerous in the queue at once.
    *   Considering the graph itself needs O(V+E) space for its adjacency list representation, the overall space is typically stated as O(V + E) for BFS.

### 7. Explain what would happen if the visited set were removed.

If the `visited` set were removed from the BFS algorithm, several critical issues would arise:

1.  **Infinite Loops:** In graphs with cycles (which our museum graph has, e.g., D -> C -> F -> D), the algorithm would get stuck in an endless loop. For example, once the path reaches D, it can go to C, then F, then back to D, and so on, repeating indefinitely without ever reaching the target or exploring new parts of the graph.

2.  **Redundant Computations and Path Exploration:** Without tracking visited nodes, the algorithm would repeatedly visit and re-process the same nodes and explore paths that have already been fully explored or are clearly not optimal. This leads to a massive increase in computations.

3.  **Incorrect Shortest Path:** While BFS intrinsically explores level by level, without a visited set, it might process a longer path to a node before a shorter one if the shorter path comes from an unvisited branch that happens to be encountered later in the queue. More importantly, it wouldn't guarantee finding *any* path in cyclic graphs, let alone the shortest.

4.  **Exponential Time and Space Complexity:** The time and space complexity would no longer be O(V + E). Instead, in the worst case (e.g., a complete graph or a graph with many cycles), it could become exponential, as the algorithm would explore every possible path combination, revisiting nodes multiple times. The queue could grow to hold an extremely large number of path states.

In essence, the `visited` set is crucial for ensuring that BFS is efficient, terminates correctly in graphs with cycles, and accurately finds the shortest path in unweighted graphs by preventing redundant work and infinite loops.